In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
import import_ipynb
from itertools import combinations
from tqdm import tqdm

In [45]:
class DataGenerator:
    def __init__(self, *args, **kwargs):
        # data params
        self.ndim = kwargs.get('ndim', 30)
        self.ndata = kwargs.get('ndata', 3000)
        
        # x params
        self.mu_x = kwargs.get('mu_x', np.zeros(self.ndim))
        cov = 0.85 * np.ones((self.ndim, self.ndim))
        np.fill_diagonal(cov, 1)
        self.cov_x = kwargs.get('cov_x', cov)
        
        # beta params
        self.num_choice = kwargs.get('num_choice', 10)
        self.mu_b = kwargs.get('mu_b', 0.0)
        self.var_b = kwargs.get('var_b', 0.4)
        
        # noise params
        self.mu_n = kwargs.get('mu_n', 0)
        self.var_n = kwargs.get('var_n', 6.25)
    
    def get_x(self):
        x = np.random.multivariate_normal(self.mu_x, self.cov_x, self.ndata)
        x = scale(x)
        x = np.concatenate((np.ones_like(x[:, 0].reshape(-1, 1)), x), axis=1)
        return x
    
    def get_beta(self):
        beta = np.zeros(self.ndim + 1)
        beta_index = np.arange(self.ndim + 1)
#         self.choice_index = np.random.choice(beta_index, self.num_choice)
        self.choice_index = np.arange(11)
        for idx in self.choice_index:
            beta[idx] = np.random.normal(self.mu_b, np.sqrt(self.var_b), 1)
        
        return beta

    def get_noise(self):
        return np.random.normal(self.mu_n, np.sqrt(self.var_n), self.ndata)
    
    def get_y(self, x, beta):
        noise = self.get_noise()
        y = x @ beta + noise
        self._get_noise_var(y, noise)
        
        return y
    
    def _get_noise_var(self, y, noise):
        self.sn = np.var(y) / np.var(noise)
    
    def generate(self):
        x = self.get_x()
        beta = self.get_beta()
        y = self.get_y(x, beta)
        return x, y, beta

In [46]:
class LSQR: 
    def __init__(self, *args, **kwargs):
        self.intercept = kwargs.get('intercept', 1)
        
    def fit(self, x, y):
        if self.intercept == 1:
            if not np.isnan(x).any():
                x = np.concatenate((np.ones_like(x[:, 0].reshape(-1, 1)), x), axis=1)
            else:
                x = np.ones_like(x.reshape(-1, 1))
            
        self.compute_gamma(x)
        self.compute_QR()
        self.beta_hat = np.linalg.inv(self.R) @ self.Q.T @ y
        
    def compute_gamma(self, x):
        self.z = np.zeros_like(x)
        self.z[:, 0] = 1.0
        self.gamma = np.zeros((x.shape[1], x.shape[1]))
        
        for i in range(x.shape[1]):
            sum_gamma_z = 0
            
            for j in range(i):
                self.gamma[j, i] = (self.z[:, j] @ x[:, i]) / (self.z[:, j] @ self.z[:, j])
                sum_gamma_z = sum_gamma_z + (self.gamma[j, i] * self.z[:, j])

            self.z[:, i] = x[:, i] - sum_gamma_z
            self.gamma[i, i] = (self.z[:, i] @ x[:, i]) / (self.z[:, i] @ self.z[:, i])
            
    def compute_QR(self):
        D = np.diag(np.linalg.norm(self.z, axis=0))
        
        self.Q = self.z @ np.linalg.inv(D)
        self.R = D @ self.gamma
        
    def predict(self, x):
        if self.intercept == 1:
            if not np.isnan(x).any():
                x = np.concatenate((np.ones_like(x[:, 0].reshape(-1, 1)), x), axis=1)
            else:
                x = np.ones_like(x.reshape(-1, 1))
        return x @ self.beta_hat

In [51]:
def scale(x):
    mean = x.mean(axis=0, keepdims=True)
    stdev = x.std(axis=0, keepdims=True, ddof=0)
    return np.divide(np.subtract(x, mean), stdev)

In [56]:
from sklearn.metrics import mean_squared_error
mse = dict()
mse_temp = []
for i in range(50):
    datagen = DataGenerator()
    x, y, beta = datagen.generate()
    lsqr = LSQR(intercept=0)
    lsqr.fit(x[:, 0].reshape(-1, 1), y)
    beta_hat = np.zeros_like(beta)
    beta_hat[0] = lsqr.beta_hat
    subs = beta_hat - beta
    mse_temp.append(mean_squared_error(beta_hat, beta))
# print(y.mean())
mse[0] = np.mean(mse_temp)

# subsets = np.arange(0, x.shape[1])

# for i, subset in enumerate(subsets):
#     if i != 31:
#         continue
#     subset = list(combinations(subsets, i + 1))
#     best_mse = 10
#     mse_temp_k = []
#     for idxs in tqdm(subset, total=len(subset), desc="training on subset {}".format(i)):
#         mse_temp = []
# #         idxs = [0] + list(idxs)
#         idxs = list(idxs)
#         print(idxs)
    
#         for j in range(50):
#             datagen = DataGenerator()
#             x, y, beta = datagen.generate()
#             x_temp = x[:, idxs].reshape(-1, len(idxs))
            
#             lsqr = LSQR(intercept=0)
#             lsqr.fit(x_temp, y)
  
#             beta_hat = np.zeros_like(beta)
#             beta_hat[idxs] = lsqr.beta_hat
#             subs = beta_hat - beta
#             mse_temp.append(np.mean((subs) ** 2))
            
#         mse_temp = np.mean(mse_temp)
#     mse[i + 1] = mse_temp
#         if mse_temp < best_mse:
#             mse[i + 1] = mse_temp
#             best_mse = mse_temp
#     if i == 0:
#         break
print(mse)

{0: -0.00463923098237486}
